In [3]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.cross_validation import train_test_split
import gc
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_validation import StratifiedKFold
import datetime

In [21]:
print('cur time = ' + str(datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S")))
train = np.load('../feature/train_fe_8_23.npy')
test = np.load('../feature/test_fe_8_23.npy')
train_labels = pd.read_csv('../feature/train_label.csv')['label'].values
print train.shape,test.shape
print('cur time = ' + str(datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S")))

cur time = 2018/08/24 20:46:42
(116624, 8847) (53093, 8847)
cur time = 2018/08/24 20:46:46


In [11]:
skf = StratifiedKFold(train_labels, n_folds=5, shuffle=True, random_state=42)
meta_train = np.zeros(shape = (len(train),6))
meta_test = np.zeros(shape = (len(test),6))

for i,(tr_ind,te_ind) in enumerate(skf):
    print 'FOLD: ',i
    print len(te_ind),len(tr_ind)
    X_train,X_train_label = train[tr_ind],train_labels[tr_ind]
    X_val,X_val_label = train[te_ind],train_labels[te_ind]
    dtrain = lgb.Dataset(X_train,X_train_label) 
    dval   = lgb.Dataset(X_val,X_val_label, reference = dtrain)   
    params = {
            'task':'train', 
            'boosting_type':'gbdt',
            'num_leaves': 15,
            'objective': 'multiclass',
            'num_class':6,
            'learning_rate': 0.05,
            'feature_fraction': 0.85,
            'subsample':0.85,
            'num_threads': 16,
            'metric':'multi_logloss',
            'seed':100
        }  
    model = lgb.train(params, dtrain, num_boost_round=100000,valid_sets=[dtrain,dval],verbose_eval=100, early_stopping_rounds=100)  
    pred_val = model.predict(X_val)
    pred_test = model.predict(test)
    
    meta_train[te_ind] = pred_val
    meta_test += pred_test
meta_test /= 5.0

FOLD:  0
23327 93297
Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.0175478	valid_1's multi_logloss: 0.0264023
[200]	training's multi_logloss: 0.00275986	valid_1's multi_logloss: 0.016544
[300]	training's multi_logloss: 0.00109042	valid_1's multi_logloss: 0.0173074
Early stopping, best iteration is:
[207]	training's multi_logloss: 0.00254555	valid_1's multi_logloss: 0.0165206
FOLD:  1
23327 93297
Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.0174761	valid_1's multi_logloss: 0.0253601
[200]	training's multi_logloss: 0.00262269	valid_1's multi_logloss: 0.0154576
Early stopping, best iteration is:
[190]	training's multi_logloss: 0.00296962	valid_1's multi_logloss: 0.0153761
FOLD:  2
23325 93299
Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.0182342	valid_1's multi_logloss: 0.0219362
[200]	training's multi_logloss: 0.00298082	valid_1's multi

In [12]:
#pd.to_pickle(meta_train,'../feature/train_meta_lgb_1.pkl')
#pd.to_pickle(meta_test,'../feature/test_meta_lgb_1.pkl')

In [4]:
train_cnn_1 = pd.read_pickle('../feature/train_meta_cnn.pkl')
test_cnn_1 = pd.read_pickle('../feature/test_meta_cnn.pkl')

In [5]:
train_lgb_1 = pd.read_pickle('../feature/train_meta_lgb_1.pkl')
test_lgb_1 = pd.read_pickle('../feature/test_meta_lgb_1.pkl')

In [6]:
train = np.hstack([train,train_cnn_1,train_lgb_1])
test = np.hstack([test,test_cnn_1,test_lgb_1])

In [7]:
train.shape,test.shape

((116624, 8859), (53093, 8859))

In [26]:
skf = StratifiedKFold(train_labels, n_folds=5, shuffle=True, random_state=42)
meta_train = np.zeros(shape = (len(train),6))
meta_test = np.zeros(shape = (len(test),6))

for i,(tr_ind,te_ind) in enumerate(skf):
    print 'FOLD: ',i
    print len(te_ind),len(tr_ind)
    X_train,X_train_label = train[tr_ind],train_labels[tr_ind]
    X_val,X_val_label = train[te_ind],train_labels[te_ind]
    dtrain = lgb.Dataset(X_train,X_train_label) 
    dval   = lgb.Dataset(X_val,X_val_label, reference = dtrain)   
    params = {
            'task':'train', 
            'boosting_type':'gbdt',
            'num_leaves': 15,
            'objective': 'multiclass',
            'num_class':6,
            'learning_rate': 0.05,
            'feature_fraction': 0.85,
            'subsample':0.85,
            'num_threads': 16,
            'metric':'multi_logloss',
            'seed':100
        }  
    model = lgb.train(params, dtrain, num_boost_round=100000,valid_sets=[dtrain,dval],verbose_eval=100, early_stopping_rounds=100)  
    pred_val = model.predict(X_val)
    pred_test = model.predict(test)
    
    meta_train[te_ind] = pred_val
    meta_test += pred_test
meta_test /= 5.0

FOLD:  0
23327 93297
Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.0132989	valid_1's multi_logloss: 0.0234082
[200]	training's multi_logloss: 0.0016545	valid_1's multi_logloss: 0.0172636
Early stopping, best iteration is:
[158]	training's multi_logloss: 0.00294376	valid_1's multi_logloss: 0.0166835
FOLD:  1
23327 93297
Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.0133569	valid_1's multi_logloss: 0.022644
[200]	training's multi_logloss: 0.00160646	valid_1's multi_logloss: 0.015774
Early stopping, best iteration is:
[165]	training's multi_logloss: 0.00259558	valid_1's multi_logloss: 0.0153602
FOLD:  2
23325 93299
Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 0.0140416	valid_1's multi_logloss: 0.0185739
[200]	training's multi_logloss: 0.00192878	valid_1's multi_logloss: 0.0103099
Early stopping, best iteration is:
[183]	training's multi_lo

In [27]:
#pd.to_pickle(meta_train,'../feature/train_meta_lgb_2.pkl')
#pd.to_pickle(meta_test,'../feature/test_meta_lgb_2.pkl')

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Lambda, Embedding, Dropout, Activation,GRU,Bidirectional
from keras.layers import Conv1D,Conv2D,MaxPooling2D,GlobalAveragePooling1D,GlobalMaxPooling1D, MaxPooling1D, Flatten
from keras.layers import CuDNNGRU, CuDNNLSTM, SpatialDropout1D
from keras.layers.merge import concatenate, Concatenate, Average, Dot, Maximum, Multiply, Subtract
from keras.models import Model
from keras.optimizers import RMSprop,Adam
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD
from keras import backend as K
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
from keras.layers import SpatialDropout1D
from keras.layers.wrappers import Bidirectional

Using TensorFlow backend.


In [12]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
train_nn = pd.DataFrame(train).fillna(0.0).values
test_nn = pd.DataFrame(test).fillna(0.0).values
ss = StandardScaler()
ss.fit(train_nn)
train_nn = ss.transform(train_nn)
test_nn = ss.transform(test_nn)

In [35]:
def MLP(FE_LEN = 8859):
    
    _input = Input(shape=(FE_LEN,),dtype='float32')
    fc = Dropout(0.25)(_input)
    fc = BatchNormalization()(fc)
    fc = Dense(256, activation='relu')(fc)
    fc = Dropout(0.25)(fc)
    fc = BatchNormalization()(fc)
    fc = Dense(512, activation='relu')(fc)
    fc = Dropout(0.25)(fc) 
    preds = Dense(6, activation = 'softmax')(fc)
    
    model = Model(inputs=_input, outputs=preds)
    model.compile(loss='categorical_crossentropy',
    optimizer='adam')
    
    return model

In [36]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
skf = StratifiedKFold(train_labels, n_folds=5, shuffle=True, random_state=42)
train_labels_nn = pd.get_dummies(train_labels).values
meta_train = np.zeros(shape = (len(train),6))
meta_test = np.zeros(shape = (len(test),6))
model_name = 'mlp_raw_feature_v1'
for i,(tr_ind,te_ind) in enumerate(skf):
    print 'FOLD: ',i
    print len(te_ind),len(tr_ind)
    X_train,X_train_label = train_nn[tr_ind],train_labels_nn[tr_ind]
    X_val,X_val_label = train_nn[te_ind],train_labels_nn[te_ind] 
    
    model_save_path = '../model_weight/{}.hdf5'.format(model_name)
    early_stopping =EarlyStopping(monitor='val_loss', patience=3)
    model_checkpoint = ModelCheckpoint(model_save_path, save_best_only=True, save_weights_only=True)
    model = MLP()
    model.fit(X_train,X_train_label,
                  validation_data=(X_val,X_val_label),
                  epochs=100,batch_size=512,
                  shuffle=True,
                  callbacks=[early_stopping,model_checkpoint]
            )
    pred_val = model.predict(X_val)
    pred_test = model.predict(test_nn)
    
    meta_train[te_ind] = pred_val
    meta_test += pred_test
meta_test /= 5.0

FOLD:  0
23327 93297
Train on 93297 samples, validate on 23327 samples
Epoch 1/100
93297/93297 [==============================] - 16s 177us/step - loss: 0.0842 - val_loss: 0.0300
Epoch 2/100
93297/93297 [==============================] - 15s 159us/step - loss: 0.0237 - val_loss: 0.0301
Epoch 3/100
93297/93297 [==============================] - 15s 158us/step - loss: 0.0191 - val_loss: 0.0290
Epoch 4/100
93297/93297 [==============================] - 15s 162us/step - loss: 0.0162 - val_loss: 0.0311
Epoch 5/100
93297/93297 [==============================] - 15s 159us/step - loss: 0.0145 - val_loss: 0.0339
Epoch 6/100
93297/93297 [==============================] - 15s 158us/step - loss: 0.0138 - val_loss: 0.0290
Epoch 7/100
93297/93297 [==============================] - 15s 160us/step - loss: 0.0128 - val_loss: 0.0312
Epoch 8/100
93297/93297 [==============================] - 15s 159us/step - loss: 0.0110 - val_loss: 0.0354
Epoch 9/100
93297/93297 [==============================] - 15s 16

In [37]:
pd.to_pickle(meta_train,'../feature/train_meta_mlp_1.pkl')
pd.to_pickle(meta_test,'../feature/test_meta_mlp_1.pkl')

In [43]:
def MLP_2(FE_LEN = 8859):
    
    _input = Input(shape=(FE_LEN,),dtype='float32')
    fc = Dropout(0.25)(_input)
    fc = BatchNormalization()(fc)
    fc = Dense(256, activation='relu')(fc)
    fc = Dropout(0.25)(fc)
    fc = BatchNormalization()(fc)
    fc = Dense(256, activation='relu')(fc)
    fc = Dropout(0.25)(fc)
    fc = BatchNormalization()(fc)
    fc = Dense(256, activation='relu')(fc)
    fc = Dropout(0.25)(fc)
    preds = Dense(6, activation = 'softmax')(fc)
    
    model = Model(inputs=_input, outputs=preds)
    model.compile(loss='categorical_crossentropy',
    optimizer='adam')
    
    return model

In [44]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
skf = StratifiedKFold(train_labels, n_folds=5, shuffle=True, random_state=42)
train_labels_nn = pd.get_dummies(train_labels).values
meta_train = np.zeros(shape = (len(train),6))
meta_test = np.zeros(shape = (len(test),6))
model_name = 'mlp_raw_feature_v2'
for i,(tr_ind,te_ind) in enumerate(skf):
    print 'FOLD: ',i
    print len(te_ind),len(tr_ind)
    X_train,X_train_label = train_nn[tr_ind],train_labels_nn[tr_ind]
    X_val,X_val_label = train_nn[te_ind],train_labels_nn[te_ind] 
    
    model_save_path = '../model_weight/{}.hdf5'.format(model_name)
    early_stopping =EarlyStopping(monitor='val_loss', patience=3)
    model_checkpoint = ModelCheckpoint(model_save_path, save_best_only=True, save_weights_only=True)
    model = MLP_2()
    model.fit(X_train,X_train_label,
                  validation_data=(X_val,X_val_label),
                  epochs=100,batch_size=512,
                  shuffle=True,
                  callbacks=[early_stopping,model_checkpoint]
            )
    pred_val = model.predict(X_val)
    pred_test = model.predict(test_nn)
    
    meta_train[te_ind] = pred_val
    meta_test += pred_test
meta_test /= 5.0

FOLD:  0
23327 93297
Train on 93297 samples, validate on 23327 samples
Epoch 1/100
93297/93297 [==============================] - 18s 198us/step - loss: 0.0928 - val_loss: 0.0321
Epoch 2/100
93297/93297 [==============================] - 15s 161us/step - loss: 0.0250 - val_loss: 0.0309
Epoch 3/100
93297/93297 [==============================] - 15s 163us/step - loss: 0.0207 - val_loss: 0.0332
Epoch 4/100
93297/93297 [==============================] - 15s 165us/step - loss: 0.0174 - val_loss: 0.0327
Epoch 5/100
93297/93297 [==============================] - 15s 160us/step - loss: 0.0163 - val_loss: 0.0332
FOLD:  1
23327 93297
Train on 93297 samples, validate on 23327 samples
Epoch 1/100
93297/93297 [==============================] - 19s 199us/step - loss: 0.1444 - val_loss: 0.0345
Epoch 2/100
93297/93297 [==============================] - 15s 161us/step - loss: 0.0272 - val_loss: 0.0283
Epoch 3/100
93297/93297 [==============================] - 15s 163us/step - loss: 0.0209 - val_loss: 0

In [45]:
pd.to_pickle(meta_train,'../feature/train_meta_mlp_2.pkl')
pd.to_pickle(meta_test,'../feature/test_meta_mlp_2.pkl')